###  Testing with SDK v2

In [ ]:
import sys
sys.path.append("../") # go to parent dir

import logging

# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential

# Enter details of your AML workspace
subscription_id = "89da9f33-fd31-4ece-861e-5fab7af4dc11"
resource_group = "mtcs-dev-aml-rg"
workspace = "mtcs-dev-aml"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

from src.ray_on_aml.core import Ray_On_AML

ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ray-on-aml-workers")

ray = ray_on_aml.getRay(num_node=2, pip_packages=["fastparquet==2022.12.0", "azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs==2022.11.2", "fsspec==2022.11.0", "tqdm"])
client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

In [ ]:
ray.available_resources()

In [ ]:
#Ray also support Ray dataset. You can read into ray dataset then convert to Dask or other ML format which is convenient for ML training.https://docs.ray.io/en/latest/data/dataset.html
# you may need to upgrade pyarrow to run this successfully
from adlfs import AzureBlobFileSystem

abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")
#if read all years and months
# data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather//", filesystem=abfs)
data =ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2015/"], filesystem=abfs)
# data.count()
# 1,584,481,119 is the count for all data 
data.show(5)

In [ ]:
ray.shutdown()
ray_on_aml.shutdown(True)

In [ ]:
ray.cluster_resources()

In [1]:
import sys
sys.path.append("../") # go to parent dir

import logging

# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential

# Enter details of your AML workspace
subscription_id = "89da9f33-fd31-4ece-861e-5fab7af4dc11"
resource_group = "mtcs-dev-aml-rg"
workspace = "mtcs-dev-aml"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

from src.ray_on_aml.core import Ray_On_AML

from azure.ai.ml import command, Input, Output
import logging


ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ray-on-aml-workers")


inputs={
        "diabetes": Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/data03022022",
        )
    }

outputs={
    "output1": Output(
        type="uri_folder",
        path="azureml://datastores/workspaceblobstore/paths/ray_outputs",
    )
}

ray = ray_on_aml.getRay(inputs = inputs, outputs=outputs, num_node=2, 
                        pip_packages=["fastparquet==2022.12.0", "azureml-mlflow==1.48.0", 
                                    "pyarrow==6.0.1", "dask==2022.2.0", "adlfs==2022.11.2", 
                                    "fsspec==2022.11.0", "tqdm"])

client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

Notice:
maxnode will be deprecated in the future. It will be moved to num_node in getRay() method.
ray[default]==2.0.0 is found in your environment.
Ray 2.0.0 version will be install in your compute cluster


Waiting cluster to start and return head node's ip
.......
 cluster is ready, head node ip  10.0.0.6


In [ ]:
ds = ray.data.range(100) 
ds.write_csv(ray_on_aml.mount_points['output1']) 

In [ ]:
df = ray.data.read_csv(ray_on_aml.mount_points['output1']) 
df.show()

In [4]:
fileName = ray_on_aml.mount_points['diabetes']
# fileName = ray_on_aml.mount_points['batch_data']
# fileName ="/mnt/batch/tasks/shared/LS_root/jobs/mtcs-dev-aml/azureml/happy_brain_j2376wy44j/wd/greenTaxi_uri/unprepared.parquet"
# fileName = "/mnt/batch/tasks/shared/LS_root/jobs/mtcs-dev-aml/azureml/eager_school_mnkf627spt/wd/diabetes_uri"

data = ray.data.read_csv(fileName).repartition(4)
data.count()

2022-12-21 02:04:20,828	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Repartition: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

442

(raylet, ip=10.0.0.8) /azureml-envs/azureml_c9f02056f99271e9bd887c91b48da60e/lib/python3.10/site-packages/ray/dashboard/agent.py:50: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.8)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.0.8) /azureml-envs/azureml_c9f02056f99271e9bd887c91b48da60e/lib/python3.10/site-packages/ray/dashboard/agent.py:470: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.8)   loop = asyncio.get_event_loop()


In [6]:
data.to_pandas()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,135
...,...,...,...,...,...,...,...,...,...,...,...
437,60,2,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,178
438,47,2,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,104
439,60,2,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,132
440,36,1,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,220


In [3]:
ray.cluster_resources()

{'object_store_memory': 4118717644.0,
 'node:10.0.0.6': 1.0,
 'memory': 8237435291.0,
 'CPU': 4.0}

In [7]:
ray.shutdown()
ray_on_aml.shutdown(True)

In [ ]:
from adlfs import AzureBlobFileSystem
#install dask, adlfs 
import dask.dataframe as dd
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
enable_dask_on_ray()
abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)

# data = ray.data.read_parquet(ray_on_aml.mount_points['ISDWeather'])

data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs)
print(data.count())

In [ ]:
client.disconnect()

###  Testing with SDK v1

In [ ]:
from azureml.core import Workspace
import sys
sys.path.append("../") # go to parent dir
import importlib

ws = Workspace.from_config()

In [ ]:
from src.ray_on_aml.core import Ray_On_AML


ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="ray-on-aml-workers", maxnode=2)
ray = ray_on_aml.getRay(ci_is_head =True, num_node=2,pip_packages=["torch==1.13.0","fastparquet==2022.12.0", "azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.12.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])


In [ ]:
ray.cluster_resources()

In [ ]:
import ray

# Load data.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from ray import train
from ray.air import session
from ray.air.config import ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer


def create_model(input_features):
    return nn.Sequential(
        nn.Linear(in_features=input_features, out_features=16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid(),
    )


def train_loop_per_worker(config):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["num_epochs"]
    num_features = config["num_features"]

    # Get the Ray Dataset shard for this data parallel worker,
    # and convert it to a PyTorch Dataset.
    train_data = train.get_dataset_shard("train")
    # Create model.
    model = create_model(num_features)
    model = train.torch.prepare_model(model)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for cur_epoch in range(epochs):
        for batch in train_data.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            # "concat_out" is the output column of the Concatenator.
            inputs, labels = batch["concat_out"], batch["target"]
            optimizer.zero_grad()
            predictions = model(inputs)
            train_loss = loss_fn(predictions, labels.unsqueeze(1))
            train_loss.backward()
            optimizer.step()
        loss = train_loss.item()
        session.report({"loss": loss}, checkpoint=TorchCheckpoint.from_model(model))


num_features = len(train_dataset.schema().names) - 1

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "batch_size": 128,
        "num_epochs": 20,
        "num_features": num_features,
        "lr": 0.001,
    },
    scaling_config=ScalingConfig(
        num_workers=2,  # Number of workers to use for data parallelism.
        use_gpu=False,
        trainer_resources={"CPU": 1},  # so that the example works on Colab.
    ),
    datasets={"train": train_dataset},
    preprocessor=preprocessor,
)
# Execute training.
result = trainer.fit()
print(f"Last result: {result.metrics}")
# Last result: {'loss': 0.6559339960416158, ...}

In [ ]:
pip install --upgrade torch

In [ ]:
ray_on_aml.shutdown(True)
ray.shutdown()